## What is a “Dynamic Page”?

A **dynamic page** is a website where some or all of the content loads **after** the page appears, often using JavaScript.

**Examples:**
- Social media feeds
- Infinite scroll
- Content that appears only after you click or scroll

---

### Problem:

- `requests` and BeautifulSoup can only see the **initial HTML**—they miss content loaded later with JavaScript.

---

### Solution:

- We use a tool that controls a real web browser, just like a human.
- **Selenium** is the most popular tool for this job.

## Handle JavaScript-rendered Content with Selenium

### What is Selenium?

**Selenium** is a Python library that can:

- Open a real browser window (like Chrome or Firefox)
- Click buttons, scroll pages, type into boxes—anything a human can do!
- Wait for content to load, then grab the new HTML

### Why use Selenium?

It lets you see and scrape the page after all the JavaScript has finished loading content.

### Setting up ChromeDriver

1. Download ChromeDriver that matches your Chrome browser version
2. Place the chromedriver file in your project folder
    - **Alternatively:** Use the full file path in your code
3. Make sure ChromeDriver is accessible in your system PATH

In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import csv

## Required Packages

- **selenium**: Controls the Chrome browser for automated web interactions
- **webdriver**: The main Selenium module for browser automation 
- **Service**: Manages the ChromeDriver service configuration
- **By**: Provides locator strategies for finding elements on a webpage
- **time**: Enables script timing and delays for page loading
- **csv**: Handles CSV file read/write operations

Each package plays a specific role in setting up automated browser interactions and data handling.


In [ ]:
# Start Chrome browser
service = Service('chromedriver.exe')  # Or full path like 'C:/path/to/chromedriver.exe'
driver = webdriver.Chrome(service=service)

base_url = "website_url_here"  # Replace with the actual URL of the book store
driver.get(base_url)
time.sleep(2)  # Wait for page to load

with open("books.csv", "w", newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Title", "Price", "Rating", "Category"])

    while True:
        books = driver.find_elements(By.CLASS_NAME, "product_pod")
        for i, book in enumerate(books, 1):
            # Title
            title = book.find_element(By.TAG_NAME, "h3").find_element(By.TAG_NAME, "a").get_attribute("title")
            # Price
            price = book.find_element(By.CLASS_NAME, "price_color").text
            # Rating
            rating_element = book.find_element(By.CLASS_NAME, "star-rating")
            rating = rating_element.get_attribute("class").split()[-1]
            # Book link (full url)
            book_link = book.find_element(By.TAG_NAME, "h3").find_element(By.TAG_NAME, "a").get_attribute("href")

            # Open the book's detail page
            driver.get(book_link)
            time.sleep(1)  # Wait for detail page to load

            # Find the category (it's in breadcrumb nav, second <a> after 'Home')
            category = driver.find_elements(By.CSS_SELECTOR, ".breadcrumb li a")[2].text

            # Write to CSV
            writer.writerow([title, price, rating, category])

            # Print progress
            print(f"Title    : {title}")
            print(f"Price    : {price}")
            print(f"Rating   : {rating}")
            print(f"Category : {category}")
            print("-" * 30)

            # Go back to the main page
            driver.back()
            # time.sleep(1)

        # After all books, check if there's a "next" button
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "li.next a")
            next_button.click()
            time.sleep(1)  # Wait for next page to load
        except:
            # No next button, break the loop
            break

driver.quit()